In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Andy Williams,Summer Knows,"Words by Marilyn and Alan Bergman, Music by Mi..."
1,Vybz Kartel,Nymphomaniac,Anju blaxx \r\nA wi luv di nymphomaniacs \r\...
2,Perry Como,It All Seems To Fall Into Line,"Something's missin', when you hold me, \r\nYo..."
3,Bob Seger,Rite Of Passage,It's the age of reason for the anarchist \r\n...
4,Nightwish,Sleepwalker,Close your eyes \r\nFeel the ocean where pass...
5,NOFX,Live Your Life,Excuses for our money \r\nIs that religion \...
6,Louis Armstrong,Sun Showers,Sun showers \r\nIt might rain \r\nSun will s...
7,Point Of Grace,Amazing,Consider your needs of the past \r\nWhen some...
8,Bon Jovi,Little Bit Of Soul,You been robbed \r\nYou been used \r\nYou be...
9,Rascal Flatts,Love Who You Love,I should have stolen every moment \r\nNow the...


In [9]:
df['text'][0]

'Words by Marilyn and Alan Bergman, Music by Michel Legrand  \r\nFrom the 1971 film, "Summer Of \'42" starring Jennifer O\'Neill  \r\nLegrand won an Oscar for his score.  \r\n  \r\nThe summer smiles, the summer knows  \r\nAnd unashamed, she sheds her clothes  \r\nThe summer smoothes the restless sky  \r\nAnd lovingly she warms the sand on which you lie  \r\nThe summer knows, the summer\'s wise  \r\nShe sees the doubts within your eyes  \r\nAnd so she takes her summer time  \r\nTells the moon to wait and the sun to linger  \r\nTwists the world \'round her summer finger  \r\nLets you see the wonder of her arms  \r\nAnd if you\'ve learned your lessons well  \r\nThere\'s little more for her to tell  \r\nOne last caress, it\'s time to dress for fall  \r\n  \r\nAnd if you\'ve learned your lesson well  \r\nThere\'s little more for her to tell  \r\nOne last caress, it\'s time to dress for fall  \r\n  \r\nTranscribed by Ronald E. Hontz  \r\nRonhontz  \r\nWorldnet.att.net  \r\n  \r\n\r\n'

In [10]:
df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [17]:
!pip install nltk



  Using cached regex-2023.12.25-cp38-cp38-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cached regex-2023.12.25-cp38-cp38-win_amd64.whl (269 kB)
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)


In [18]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [19]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.        , 0.01540924, 0.02834043, ..., 0.04213374, 0.02532195,
       0.02923836])

In [23]:
df[df['song'] == 'Friends Unknown']

,artist,song,text


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [28]:
recommendation('Rite Of Passage')

['Nine Tonight',
 'Tonight',
 'Died In Your Arm Tonight',
 'When Love Takes Over',
 'Tonight Is Forever',
 'After Tonight',
 'Tonight',
 "Let's Make Tonight Special",
 'Tonight',
 'Land Of Canaan',
 'Real Cool Time',
 "She Don't Want Nobody Near",
 'Wasted White Boys',
 'Almost In Love',
 'I Wanna Make You Happy',
 'Long Tall Sally',
 'Speed Of Light',
 'Heaven Tonight',
 "I Won't Be Lonely Tonight",
 'Need Your Love Tonight']

In [29]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))